In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import pandas as pd
import os

In [2]:
# Đường dẫn đến tệp JSON credential của bạn
credential_path = 'key.json'

# Phạm vi (scopes) cho Google Sheets API
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

# Xác thực bằng JSON credential và scope
credentials = ServiceAccountCredentials.from_json_keyfile_name(credential_path, scope)
gc = gspread.authorize(credentials)
drive_service = build('drive', 'v3', credentials=credentials)

In [3]:
# Tải file dữ liệu mới lên Google Drive
def upload_and_convert_to_gsheet(file_path: str, parents: list = None):
    if not os.path.exists(file_path):
        print(f'{file_path} not found.')
        return None
    try:
        file_metadata = {
            'name': os.path.basename(file_path).replace('.csv', ''),
            'mimeType': 'application/vnd.google-apps.spreadsheet',
            'parents': parents                                         
        }
        
        media = MediaFileUpload(filename=file_path, mimetype='text/csv')
        
        response = drive_service.files().create(
            media_body=media,
            body=file_metadata
        ).execute()
        
        print(f"Đã tải file {file_path} lên Google Drive với ID: {response['id']}")
        return response['id']
    except Exception as e:
        print(f"Lỗi tải file {file_path}: {e}")
        return None

# Gộp dữ liệu mới vào Google Sheet tổng hợp
def add_new_year_data(file_path, year, target_spreadsheet_id):
    file_id = upload_and_convert_to_gsheet(file_path, parents=['1zuHsJ1bFvl2nxlMYvXw_QP8sHbeyeobt'])
    if not file_id:
        print(f"Không thể tải file {file_path} lên Google Drive.")
        return
    
    sheet_name = f'data_{year}'
    source_sheet = gc.open_by_key(file_id).sheet1
    target_data = source_sheet.get_all_values()

    target_sheet = gc.open_by_key(target_spreadsheet_id)
    try:
        target_worksheet = target_sheet.worksheet(sheet_name)
    except gspread.exceptions.WorksheetNotFound:
        target_worksheet = target_sheet.add_worksheet(sheet_name, rows=len(target_data), cols=len(target_data[0]))

    target_worksheet.update('A1', target_data)
    print(f"Dữ liệu của năm {year} đã được thêm vào {target_spreadsheet_id}")

# ID của file Google Sheets tổng hợp
target_spreadsheet_id = '1z-eAvFC5g_425HfZJUwUxB6sKLFA7iHiruZN653jYbM'

# Ví dụ thêm dữ liệu của năm 2024
add_new_year_data('D:/TTTN/CODE/Google API/Data Update/data_2024.csv', 2024, target_spreadsheet_id)

Uploaded and converted D:/TTTN/CODE/Google API/DATA UPDATE/data_2024.csv to Google Sheets with ID: 18mpgEWvVVXk0WZcsX_p-xUeZ0w7NjbMoQFjqJCjXBTo


C:\Users\fna\AppData\Local\Temp\ipykernel_3008\2938849886.py:43: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  target_worksheet.update('A1', target_data)


Dữ liệu của năm 2024 đã được thêm vào 1z-eAvFC5g_425HfZJUwUxB6sKLFA7iHiruZN653jYbM
